# Dashboards

In [1]:
import numpy as np
import pandas as pd
import scipy.stats

import bokeh.io
import bokeh.layouts
import bokeh.models
import bokeh.plotting

notebook_url = 'localhost:8888' # Check browser, need to specify for output dashboard
bokeh.io.output_notebook()

Loading BokehJS ...

In [2]:
mu = 0.0
sigma = 1.0

x = np.linspace(-10, 10, 400)
pdf = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

source = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf)) # important to specify

p1 = bokeh.plotting.figure(
    frame_width=350,
    frame_height=200,
    x_axis_label='x',
    y_axis_label='f(x)',
    x_range=[-10, 10],
)

p1.line(source=source, x='x', y='pdf', line_width=2)

bokeh.io.show(p1)
# We will not show it because if it is in a dashboard, a given plot can only
# be shown there in a notebook. Instead, it's displayed as an image below. cf. p1

In [3]:
mu = 0.0
sigma = 1.0

x = np.linspace(-10, 10, 400)
pdf = scipy.stats.norm.pdf(x, loc=mu, scale=sigma)

source = bokeh.models.ColumnDataSource(dict(x=x, pdf=pdf)) # important to specify

p = bokeh.plotting.figure(
    frame_width=350,
    frame_height=200,
    x_axis_label='x',
    y_axis_label='f(x)',
    x_range=[-10, 10],
)

p.line(source=source, x='x', y='pdf', line_width=2)

# We will not show it because if it is in a dashboard, a given plot can only
# be shown there in a notebook. Instead, it's displayed as an image below.

GlyphRenderer(id='1143', ...)

In [4]:
mu_slider = bokeh.models.Slider(title='µ', start=-5.0, end=5.0, step=0.1, value=0.0, width=100)
sigma_slider = bokeh.models.Slider(title='σ', start=+0.1, end=5.0, step=0.1, value=0.0, width=100)

# see picture
# option + m = mu
# can add grec keyboard + shortcut for letter to appear
# No need to change all aspect figures, can decide what to update

In [5]:
def norm_callback(attr, old, new):
    mu = mu_slider.value
    sigma = sigma_slider.value
    
    pdf = scipy.stats.norm.pdf(source.data['x'], loc=mu, scale=sigma) 
    #source has many attribute, here we specify which attribute we want to change.
    
    source.data['pdf'] = pdf

In [6]:
# We need to tell the slider what it needs to excute

mu_slider.on_change('value',norm_callback)
sigma_slider.on_change('value',norm_callback)

In [7]:
# Now the figure, the plot, the slider and code they have to execute are created.
# We need to create the slider graphically

# positions, spaces, etc.
slider_layout = bokeh.layouts.column(
    bokeh.layouts.Spacer(height=30),
    mu_slider,
    bokeh.layouts.Spacer(height=15),
    sigma_slider
)

norm_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    slider_layout
)

In [8]:
# Now everything is created. We need to plot it to show, for that needs to create an app.

def norm_app(doc):
    doc.add_root(norm_layout)

In [10]:
bokeh.io.show(norm_app, notebook_url=notebook_url)

In [11]:
# Many different possible widgets ! Check documentation !

In [12]:
df = pd.read_csv('data/gfmt_sleep.csv', na_values='*')
df['insomnia'] = df['sci'] <= 16

df.head()

,participant number,gender,age,correct hit percentage,correct reject percentage,percent correct,confidence when correct hit,confidence when incorrect hit,confidence when correct reject,confidence when incorrect reject,confidence when correct,confidence when incorrect,sci,psqi,ess,insomnia
0,8,f,39,65,80,72.5,91.0,90.0,93.0,83.5,93.0,90.0,9,13,2,True
1,16,m,42,90,90,90.0,75.5,55.5,70.5,50.0,75.0,50.0,4,11,7,True
2,18,f,31,90,95,92.5,89.5,90.0,86.0,81.0,89.0,88.0,10,9,3,True
3,22,f,35,100,75,87.5,89.5,NaN,71.0,80.0,88.0,80.0,13,8,20,True
4,27,f,74,60,65,62.5,68.5,49.0,61.0,49.0,65.0,49.0,13,9,12,True


In [13]:
# Use dashboard to explore new data...
# First step, pen and paper, sketch
# see picture insect tracking "coleoseum"
# for sleep too, see paper notes

In [14]:
xy_options = list(
    df.columns[~df.columns.isin(['participant number', 'gender', 'insomnia'])]
)

x_selector = bokeh.models.Select(
    title='x', options=xy_options, value='percent correct', width=200
)

y_selector = bokeh.models.Select(
    title='y', options=xy_options, value='confidence when correct', width=200
)

colorby_selector = bokeh.models.Select(
    title='color by', options=['none', 'gender', 'insomnia'], value='confidence when correct', width=200
)


In [15]:
%load_ext blackcellmagic

In [16]:
source = bokeh.models.ColumnDataSource(
    dict(x=df[x_selector.value], y=df[y_selector.value], gender=df['gender'], insomnia=df['insomnia'])
)

source.data["color"] = ["#1f77b3"] * len(df)  # bokeh blue

In [17]:
p = bokeh.plotting.figure(
    frame_height=250,
    frame_width=250,
    x_axis_label=x_selector.value,
    y_axis_label=y_selector.value,
    tooltips=[
        ('gender', '@gender'),
        ('insomnia', '@insomnia') #do not forget to add to source otherwise ??? in the plot cursor 
    ]
)

circle = p.circle(source=source, x="x", y="y", color="color")

In [18]:
def gfmt_callback(attr, old, new):
    if colorby_selector.value == 'none':
        source.data['color'] = ['#1f77b3'] * len(df)
    elif colorby_selector.value == 'gender':
        source.data['color'] = [
            '#1f77b3' if gender == 'f' else '#ff7e0e'
            for gender in df['gender']
        ]
    elif colorby_selector.value == 'insomnia':
        source.data['color'] = [
            '#ff7e0e' if insomnia else '#1f77b3'
            for insomnia in df['insomnia']
        ]
            
    source.data['x'] = df[x_selector.value]
    p.xaxis.axis_label = x_selector.value
        
    source.data['y'] = df[y_selector.value]
    p.yaxis.axis_label = y_selector.value

In [19]:
# Can have a widget toggle button to reset slider value to default or the one from paper.

In [20]:
# we need to connect callback to the widget

colorby_selector.on_change('value', gfmt_callback)
x_selector.on_change('value', gfmt_callback)
y_selector.on_change('value', gfmt_callback)

In [21]:
# We do the layout
# 15 pixels

gfmt_layout = bokeh.layouts.row(
    p,
    bokeh.layouts.Spacer(width=15),
    bokeh.layouts.column(
        x_selector,
        bokeh.layouts.Spacer(height=15),
        y_selector,
        bokeh.layouts.Spacer(height=15),
        colorby_selector,
    ),
)

In [22]:
# We do the app
def gfmt_app(doc):
    doc.add_root(gfmt_layout)

In [24]:
bokeh.io.show(gfmt_app, notebook_url=notebook_url)

In [27]:
# Needs python to run
# We want for people without python to be able to interact with the dashboard
# when send file we want to work straight from the browser (javascript) with needing python
# if callsback in java script faster, so very smooth.